# Demo 3 - Monitoring & Tracing Example, and Model Differences
By: [Lior Gazit](https://github.com/LiorGazit).  
Repo: [Agents-Over-The-Weekend](https://github.com/PacktPublishing/Agents-Over-The-Weekend/tree/main/Lior_Gazit/workshop_september_2025/)  
Running LLMs locally for free: This code leverages [`LLMPop`](https://pypi.org/project/llmpop/) that is dedicated to spinning up local or remote LLMs in a unified and modular syntax.  

<a target="_blank" href="https://colab.research.google.com/github/PacktPublishing/Agents-Over-The-Weekend/blob/main/Lior_Gazit/workshop_september_2025/codes_for_Lior_Bootcamp_talk_sept2025_demo3.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a> (pick a GPU Colab session for fastest computing)  

```
Disclaimer: The content and ideas presented in this notebook are solely those of the author, Lior Gazit, and do not represent the views or intellectual property of the author's employer.
```

Installing:

In [1]:
%pip -q install llmpop
%pip -q install langchain langchain_core tiktoken langsmith langchain_openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


**Imports:**

In [2]:
import os
import requests

Code example using LangSmiths's trace support:

In [3]:
import os
from getpass import getpass
import openai

# Set up environment variables (make sure your keys are set correctly)
if "langchain_api_key" not in globals():
  langchain_api_key = getpass("Paste your LangChain API key: ")
if not openai.api_key:
  openai.api_key = getpass("Paste your OpenAI API key: ")

os.environ["LANGSMITH_TRACING"]="true"
os.environ["LANGSMITH_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = langchain_api_key
os.environ["OPENAI_API_KEY"] = openai.api_key
# IMPORTANT: If you change the designated project, you must restart the notebook kernel.
os.environ["LANGCHAIN_PROJECT"] = "multi-agent-demo05"

import time
from langchain_openai import ChatOpenAI
from langsmith import traceable
from langsmith.run_helpers import trace
from langchain_core.prompts import ChatPromptTemplate
import tiktoken

# Helper to count tokens
def count_tokens(text, encoding_name="cl100k_base"):
    enc = tiktoken.get_encoding(encoding_name)
    return len(enc.encode(text))

# Setup LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Prompts for agents
coder_prompt = ChatPromptTemplate.from_template(
    "You are a coding assistant. Write concise Python code for this task:\n{task}"
)

reviewer_prompt = ChatPromptTemplate.from_template(
    "You are a meticulous code reviewer. Identify bugs or improvements in the following code:\n{code}"
)

# Chains
coder_chain = coder_prompt | llm
reviewer_chain = reviewer_prompt | llm

# Traceable agent function
@traceable(name="multi_agent_interaction")
def multi_agent_interaction(task):
    # Coder
    start_coder = time.time()
    coder_response = coder_chain.invoke({"task": task})
    coder_duration = time.time() - start_coder
    coder_code = coder_response.content

    print(f"\n=== Coder Output (Time: {coder_duration:.2f}s, Tokens: {count_tokens(coder_code)}) ===\n")
    print(coder_code)

    # Reviewer
    start_reviewer = time.time()
    reviewer_response = reviewer_chain.invoke({"code": coder_code})
    reviewer_duration = time.time() - start_reviewer
    reviewer_feedback = reviewer_response.content

    print(f"\n=== Reviewer Feedback (Time: {reviewer_duration:.2f}s, Tokens: {count_tokens(reviewer_feedback)}) ===\n")
    print(reviewer_feedback)

# Execute with trace context
with trace("multi_agent_demo_run"):
    print("\nStarting tracing for project <" + os.environ["LANGCHAIN_PROJECT"] + ">, funtion <multi_agent_interaction>")
    task_description = "Write a Python function `reverse_string(s)` that returns the reverse of the string."
    print(f"\nTask for coder to perform:\n{task_description}")
    multi_agent_interaction(task_description)


Paste your LangChain API key:  ········
Paste your OpenAI API key:  ········



Starting tracing for project <multi-agent-demo05>, funtion <multi_agent_interaction>

Task for coder to perform:
Write a Python function `reverse_string(s)` that returns the reverse of the string.

=== Coder Output (Time: 1.99s, Tokens: 41) ===

Certainly! Here's a concise Python function to reverse a string:

```python
def reverse_string(s):
    return s[::-1]
```

This function uses Python's slicing feature to reverse the string.

=== Reviewer Feedback (Time: 8.83s, Tokens: 379) ===

The provided Python function `reverse_string` is a concise and efficient way to reverse a string using Python's slicing feature. However, there are a few considerations and potential improvements that could be made, depending on the context in which this function is used:

1. **Input Validation**: 
   - If the function is expected to handle only strings, you might want to add a check to ensure that the input is indeed a string. This can help prevent unexpected behavior or errors if the function is calle

Code example for where building the logging process ourselves:

In [4]:
import time
import logging
import json
import tiktoken
from llmpop import init_llm


# 1. Basic logging setup
logging.basicConfig(level=logging.INFO, format="%(message)s")

# 2. Helper: count tokens using tiktoken
def count_tokens(text, encoding_name="cl100k_base"):
    enc = tiktoken.get_encoding(encoding_name)
    return len(enc.encode(text))

# 3. Helper: log each call
def log_call(step_name, prompt, response, start, end, log_file="llm_trace.log"):
    record = {
        "step": step_name,
        "prompt_tokens": count_tokens(prompt),
        "response_tokens": count_tokens(response),
        "duration_s": round(end - start, 3),
        "timestamp": start
    }
    # Console output
    logging.info(f"[{step_name}] {record['duration_s']}s | "
                 f"prompt_tokens={record['prompt_tokens']} | "
                 f"response_tokens={record['response_tokens']}")
    # Append to JSON‑lines file
    with open(log_file, "a") as f:
        f.write(json.dumps(record) + "\n")

# 4. Load your model (local Ollama example)
model = init_llm(model="CodeLlama", provider="ollama", verbose=False)

# 5. Step 1: Coder agent (generate code)
step1_prompt = "Write a Python function `reverse_string(s)` that returns the reverse of s."
start = time.time()
step1_response = model.invoke(step1_prompt)
end = time.time()
log_call("Coder", step1_prompt, step1_response.content, start, end)

# 6. Step 2: Reviewer agent (review code)
step2_prompt = f"Review this code for correctness and edge cases:\n\n{step1_response}"
start = time.time()
step2_response = model.invoke(step2_prompt)
end = time.time()
log_call("Reviewer", step2_prompt, step2_response.content, start, end)

# 7. Print outputs
print("=== Coder’s Code ===\n", step1_response.content)
print("\n=== Reviewer’s Feedback ===\n", step2_response.content)

# 8. Inspect the log file if desired:
print("\n---\nPrinting the log:")
!head -n 10 llm_trace.log

RuntimeError: Ollama server at http://127.0.0.1:11434 failed to start within 15s.